For Aassigment 5 Machine Translation: IBM Model 1

In [17]:
# write code to collect statistics about word frequencies in the two languages. 
# Print the 10 most frequent words in each language.
# using a Counter
from collections import Counter
import re

def get_words(filename):
    with open(filename, 'r') as f:
        text = f.read()
    words = re.findall(r'\w+', text.lower())
    return words

def get_10_most_frequent_words():
    english_words = get_words('./dat410_europarl/europarl-v7.fr-en.lc.en')
    french_words = get_words('./dat410_europarl/europarl-v7.fr-en.lc.fr')
    english_word_counts = Counter(english_words)
    french_word_counts = Counter(french_words)
    print('English:', english_word_counts.most_common(10))
    print('French:', french_word_counts.most_common(10))

get_10_most_frequent_words()

# print the probabitly of "zebra" in the two languages
english_words = get_words('./dat410_europarl/europarl-v7.fr-en.lc.en')
english_word_counts = Counter(english_words)
print("English:%f%%" % (english_word_counts['speaker'] / len(english_words) * 100)) 
print("English:%f%%" % (english_word_counts['zebra'] / len(english_words) * 100))

English: [('the', 19627), ('of', 9534), ('to', 8992), ('and', 7214), ('in', 6197), ('is', 4453), ('that', 4421), ('a', 4388), ('we', 3341), ('this', 3332)]
French: [('apos', 16729), ('de', 14528), ('la', 9746), ('et', 6620), ('l', 6536), ('le', 6177), ('à', 5588), ('les', 5587), ('des', 5232), ('que', 4797)]
English:0.004571%
English:0.000000%


In [79]:
# bigram language model

from math import log,exp

class BigramLanguageModel:
    def __init__(self, words):
        self.bigrams = Counter(zip(words, words[1:]))
        self.unigrams = Counter(words)
        self.total = len(words)

    def probability_w(self, word1, word2):
        if self.unigrams[word1] == 0 \
            or self.unigrams[word2] ==0 \
            or self.bigrams[(word1,word2)] == 0:
            # if word non exist
            return 1.0 / (self.total + 1)
        return self.bigrams[(word1,word2)]/self.unigrams[word1]
    
    def calculate_p_s(self, words: list):
        # length of words should be at least 1
        if len(words) == 0: return 1
        Ps=1.0 # the probability_w (words[0]|<start>) is 1
        for i in range(1,len(words)):
            pt = self.probability_w(words[i-1],words[i])
            Ps = Ps* pt
            # print("P(%s|%s) is %.15f \t P(S): %.15f" %(words[i],words[i-1], pt, Ps))
            # print("\t (%s, %s):%d, (%s): %d" %(words[i-1],words[i],
            #                                    self.bigrams[(words[i-1],words[i])], 
            #                                    words[i-1],
            #                                    self.unigrams[words[i-1]]))
        return Ps
    
    def calculate_p_s_log(self, words: list):
        # length of words should be at least 1
        if len(words) == 0: return 1
        Ps=0.0 # the probability_w (words[0]|<start>) is 1
        for i in range(1,len(words)):
            pt = log(self.probability_w(words[i-1],words[i]))
            Ps = Ps + pt
            # print("P(%s|%s) is %.15f \t P(S): %.15f" %(words[i],words[i-1], pt, Ps))
            # print("\t (%s, %s):%d, (%s): %d" %(words[i-1],words[i],
            #                                    self.bigrams[(words[i-1],words[i])], 
            #                                    words[i-1],
            #                                    self.unigrams[words[i-1]]))
        return exp(Ps)

En_bigramM = BigramLanguageModel(english_words)

# print(english_words[:100])

# debug
# En_bigamM.probability_w('please','open') 
# En_bigamM.bigrams.most_common(10)
# 

# case 1 normal short sentence "the house rose", from dataset
print("----- Normal case")
sentence = "the house rose"
print("P('%s') is %.15f, lP: %.15f " % (sentence, 
                En_bigramM.calculate_p_s(sentence.split(' '))), 
                En_bigramM.calculate_p_s_log(sentence.split(' '))) ) 
print("-----")
sentence = "it says that this should be done"
print("P('%s') is %.15f " % (sentence, En_bigramM.calculate_p_s(sentence.split(' '))) ) 

# case 2 words non exist
print("----- Non exist")
sentence = "the house rose NNN"
print("P('%s') is %.15f " % (sentence, En_bigramM.calculate_p_s(sentence.split(' '))) ) 

# case 3 words super length
print("----- over length")
sentence = "you will be aware from the press and television that there have been a number of bomb explosions and killings in sri lanka"
print("len(%d) ,P('%s ...') is %.15f " % (len(sentence.split(" ")),sentence[:10], En_bigramM.calculate_p_s(sentence.split(' '))) ) 


----- Normal case
P('the house rose') is 0.000013543729801 
-----
P('it says that this should be done') is 0.000000000931698 
----- Non exist
P('the house rose NNN') is 0.000000000051588 
----- over length
len(23) ,P('you will b ...') is 0.000000000000000 
